In [ ]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from scipy.signal import TransferFunction, lsim, butter
from scipy.fft import fft, ifft

# Requirements
Interesting signal f_i =< 8 kHz, A_i =< 1
Noise signal f_n >= 11 kHz, A_n =< 1
No signal between 8 kHz to 11 kHz

# Task
Design a anti-aliasing filter (LP) followed by 12-bit AD converter sampling at f_s <= 24 kHz
Minimize order of anti-aliasing filter
Interference in final signal must not exceed 0.5/2^11 V. 
Maximum ripple 3 dB

# Thoughts
The minimal sampling rate f_s  > 2*beta, where beta is the maximum interesting frequency
Thus we need f_s > 16 kHz
Let's start with f_s = 17 kHz

Let's start by calculating the order of the Butterworth filter 
The order N is derived by: 
N = 1/2 * log(G_p / G_s) / log(w_p/w_s)
Where:
G_p is the gain of the pass-band: G_p = 1/(1-δ_p)^2 - 1
G_s is the gain of the stop-band: G_s = 1/δ_s^2 -1 
Note that δ_p is the maximum ripple/deviation in the pass-band and δ_s is the maximum ripple/deviation in the stop-band

w_p is the maximum frequency of the pass-band
w_s is the minimum frequency of the stop-band 
We choose w_p 8 kHz and w_s 11 kHz and caluclate the order:

In [ ]:
w_p, w_s = 8000, 11000
# A_db = 20*log(A1/A2) -> A1 = A2*10^(A_db/20)
δ_p = 10**(3/20)  # 1.4125375446227544
δ_s = 0.5/2**11 # maximum interference allowed by specification
G_p = 1/(1-δ_p)**2 - 1 #4.875881669435278
G_s = 1/δ_s**2 -1  #
N = 1/2 * np.log(G_p/G_s)/np.log(w_p/w_s)
print(N) # We need N >= 24

23.63173965273065


Let's also calculate the order needed for a Chebyschev filter 

In [ ]:
order = 2
b_filter = butter(order, 8000,'lowpass', analog=True)

In [ ]:
# w = 2*pi*f
f_s = 17000
T_s = 1/f_s
t = np.linspace(0, 15*T_s, 100000)

def x1(t):
    return np.sin(2*np.pi*1000*t) # 1 kHz

def x2(t):
    return np.sin(2*np.pi*12000*t) # 12 kHz

# Raw signals
plt.plot(t, x1(t), label="1 kHz signal")
plt.plot(t, x2(t), label="12 kHz signal")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude [V]")
plt.legend()
plt.show()

In [47]:
# Sampled signals
t1 = np.linspace(0, 15*T_s, 15) # 15 samples on a time of 15 sample periods -> sampling
x1_s = x1(t1)
x2_s = x2(t1)

plt.plot(t1, x1_s, label="1 kHz signal")
plt.plot(t1, x2_s, label="12 kHz signal")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude [V]")
plt.legend()
plt.show()